In [11]:
import httplib2
import oauth2
import urllib3
import types
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gender_detector import GenderDetector 
import psycopg2, psycopg2.extras
from causalinference import CausalModel
from causalinference.utils import random_data
import httplib
import base64
import json # For Microsoft Face API
import urllib as urllib # For Microsoft Face API
import time 
import csv
import datetime 

class UpworkDataFormatter:
    
    def __init__(self):
        # Settings
        self.present_date = "11/2017" # This is the month in which the data was collected
        self.all_data_file_name = '2017_11_01_upwork_unitedstates_allskills_nowork.csv' # Filename for all data
        self.gender_data_file_name = '11_29_2017_upwork_gender_analysis_unitedstates_allskills.csv' # Filename for gender data
        self.data_log_file_name = 'log_2017_11_01__upwork_unitedstates_allskills_nowork.txt'
        self.gender_log_file_name = "log_upwork_identify_gender_2017_11_29_unitedstates_allskills.txt"
        self.github_photo_path = 'https://raw.githubusercontent.com/efoongch/upwork-pay-by-gender/master/resized_profile_photos/resized_images_unitedstates_allskills_2017_11_01/'
        
        # Write a log
        self.log = open(self.data_log_file_name, 'a')
        self.photo_log = open(self.gender_log_file_name, 'a')
        self.log.write("We have started analyzing data!" + "\n")
        self.log.flush()
        self.photo_log.write("We have started identifying gender in photos!" + "\n")
        self.photo_log.flush()

        # Connect to the database 
        self.conn = psycopg2.connect("dbname=eureka01")
        self.cur = self.conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
        psycopg2.extensions.register_adapter(dict, psycopg2.extras.Json)
        
        # Get detailed_info from workers in our database
        self.cur.execute("SELECT detailed_info FROM upwork_unitedstates_allskills_2017_11_01;")
        
        # Initialize arrays for Causal Analysis 
        self.user_count = 1
        self.bill_rate_array = []
        self.gender_array = []
        self.all_covariates_array = []
        self.country_id_list = []
        self.job_category_id_list = []
        self.education_id_list = []
        self.error_country_id = 0
        self.error_education_id = 0
        self.error_job_category_id = 0
    
    def save_all_to_csv(self):
        with open(self.all_data_file_name, 'w') as csvfile:
            fieldnames = ['user_count','worker_id', 'first_name', 'bill_rate', 
                          'country', 'country_id', 'degree', 'education', 'education_id', 
                          'job_category', 'job_category_id']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            
            for user in self.cur:
                try: 
                    user_count = self.user_count
                    worker_id = user[0]["ciphertext"]
                    print "-----" + worker_id + "------"
                    first_name = user[0]["dev_first_name"].encode('utf-8').strip()
                    bill_rate = user[0]["dev_bill_rate"]
                    country = user[0]["dev_country"]
                    country_id = self.encode_category(country, self.country_id_list)
                    degree = self.show_degree(user)
                    education = self.calculate_education(user)
                    education_id = self.encode_category(education, self.education_id_list)
                    job_category = self.identify_job_category(user)
                    job_category_id = self.encode_category(job_category, self.job_category_id_list)
            
                    writer.writerow({'user_count': user_count, 'worker_id': worker_id, 'first_name': first_name,
                                 'bill_rate': bill_rate, 'country': country, 'country_id': country_id, 'degree': degree, 'education': education, 
                                 'education_id': education_id, 'job_category': job_category, 'job_category_id': job_category_id})
                except:
                    print "Ran into some error at user {0}".format(self.user_count)
                    print json.dumps(user[0], indent=2)
                    
                    writer.writerow({'user_count': user_count, 'worker_id': "error", 'first_name': "error",
                                 'bill_rate': "error", 'country': "error", 'country_id': self.error_country_id, 'degree': "error", 'education': "error", 
                                 'education_id': self.error_education_id, 'job_category': "error", 'job_category_id': self.error_job_category_id})
                    
                self.user_count += 1
                
            '''
            for user in self.cur:
                try: 
                    user_count = self.user_count
                    worker_id = user[0]["ciphertext"]
                    first_name = user[0]["dev_first_name"].encode('utf-8').strip()
                    bill_rate = user[0]["dev_bill_rate"]
                    country = user[0]["dev_country"]
                    country_id = self.encode_category(country, self.country_id_list)
                    degree = self.show_degree(user)
                    education = self.calculate_education(user)
                    education_id = self.encode_category(education, self.education_id_list)
                    work_experience = self.calculate_work_experience(user)
                    job_category = self.identify_job_category(user)
                    job_category_id = self.encode_category(job_category, self.job_category_id_list)
            
                    writer.writerow({'user_count': user_count, 'worker_id': worker_id, 'first_name': first_name,
                                 'bill_rate': bill_rate, 'country': country, 'country_id': country_id, 'degree': degree, 'education': education, 
                                 'education_id': education_id, 'work_experience': work_experience, 'job_category': job_category, 'job_category_id': job_category_id})
                except:
                    print "Ran into some error at user {0}".format(self.user_count)
                    print json.dumps(user[0], indent=2)
                    
                    writer.writerow({'user_count': user_count, 'worker_id': "error", 'first_name': "error",
                                 'bill_rate': "error", 'country': "error", 'country_id': self.error_country_id, 'degree': "error", 'education': "error", 
                                 'education_id': self.error_education_id, 'work_experience': "error", 'job_category': "error", 'job_category_id': self.error_job_category_id})
                    
                self.user_count += 1
                '''

    def save_gender_to_csv(self): # Identify gender of users in photos that have already been pushed to GitHub 
        with open(self.gender_data_file_name, 'w') as csvfile:
            fieldnames = ['user_count','worker_id', 'gender']
            gender_writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            gender_writer.writeheader()
            
            for user in self.cur:
                try: 
                    user_count = self.user_count
                    worker_id = user[0]["ciphertext"]
                    gender = self.identify_gender()
     
                    gender_writer.writerow({'user_count': user_count, 'worker_id': worker_id, 'gender': gender})
        
                except Exception as err:
                    user_count = self.user_count
                    print "Ran into some error saving gender to csv at {0}".format(self.user_count)
                    self.photo_log.write("Ran into an error saving gender to csv at {0}: {1}".format(self.user_count, err) + "\n")
                    self.photo_log.flush()
                    gender_writer.writerow({'user_count': user_count, 'worker_id': "error", 'gender': "error"})
                    
                self.user_count += 1
        
    def identify_gender(self): # Returns gender as a string
        if (self.user_count%10 == 0): # Set timeout for rate limiting; paid subscription up to 10 per second 
            time.sleep(1)
        
        try: 
            print "Recognizing Face Number " + str(self.user_count)
            raw_face_data = self.recognize_faces()
            
            if(len(raw_face_data) == 0): 
                return "unidentified"
            
            else:
                gender = raw_face_data[0]["faceAttributes"]["gender"]
                return str(gender)
            
        except Exception as err:
            print err
            self.photo_log.write("Ran into an error in identifying gender at {0}: {1}".format(self.user_count, err) + "\n")
            self.photo_log.flush()
            return "error"
        
    def recognize_faces(self):
        # Replace the subscription_key string value with your valid subscription key.
        subscription_key = 'd6ef52f1ff334ace97271267626f11bc' # This is key 1 for the paid subscription 

        # Replace or verify the region.
        uri_base = 'eastus2.api.cognitive.microsoft.com'

        # Request headers.
        headers = {
            'Content-Type': 'application/json',
            'Ocp-Apim-Subscription-Key': subscription_key,
        }

        # Request parameters.
        params = urllib.urlencode({
            'returnFaceId': 'true',
            'returnFaceLandmarks': 'false',
            'returnFaceAttributes': 'age,gender,headPose,smile,facialHair,glasses,emotion,hair,makeup,occlusion,accessories,blur,exposure,noise',
        })

        # The URL of a JPEG image to analyze.
        beg_body = "{'url':"
        end_body = "}"
        path = self.github_photo_path + str(self.user_count) + ".jpg"
        body = beg_body + '"' + path + '"' + end_body
        
        try:
            # Execute the REST API call and get the response.
            conn = httplib.HTTPSConnection('eastus2.api.cognitive.microsoft.com')
            conn.request("POST", "/face/v1.0/detect?%s" % params, body, headers)
            response = conn.getresponse()
            data = response.read()
            parsed = json.loads(data)
            return parsed
            
        except Exception as e:
            print("[Errno {0}] {1}".format(e.errno, e.strerror))
            return "error"
            
        '''
        # 'data' contains the JSON data. The following formats the JSON data for display.
        parsed = json.loads(data)
        print ("Response:")
        print (json.dumps(parsed, sort_keys=True, indent=2))
        conn.close()
        '''
    
    def calculate_work_experience(self, user):
        print "CALCULATING WORK EXPERIENCE"
        total_experience = 0
        work_experience_array = []
        
        try: 
            work_experience_list = user[0]["experiences"]["experience"]
            print "Were able to find work experiences"
            
        except:
            print "We did not find work experiences"
            return "none"
        
        if (type(work_experience_list) is list):
            count = 0
            total_overlap = 0
            for experience in work_experience_list:
                start_date = experience["exp_from"]
                end_date = experience["exp_to"]
                if (end_date == "Present"):
                    end_date = self.present_date
                start_datetime = datetime.datetime.strptime(start_date, "%m/%Y")
                end_datetime = datetime.datetime.strptime(end_date, "%m/%Y")
                
                
                if (start_datetime >= end_datetime): #If the dates are somehow flipped
                    return "error"
                    print "The dates were flipped or time was too short"
                    
                else: 
                    work_experience_array.append([start_datetime, end_datetime])
                    experience_duration = (end_datetime - start_datetime).days
                    print "Experience duration: {0}".format(experience_duration)
                    total_experience += experience_duration
                
                print "Work experience array: {0}".format(work_experience_array)
                print "Work experience array length: {0}".format(len(work_experience_array))
                print "Total experience before overlap: {0}".format(total_experience)
            
            while (count < len(work_experience_array) - 1): # Check for overlap in work experience 
                start_newer = work_experience_array[count][0]
                print "Start newer: {0}".format(start_newer)
                end_newer = work_experience_array[count][1]
                print "End newer: {0}".format(end_newer)
                start_older = work_experience_array[count+1][0]
                print "Start older: {0}".format(start_older)
                end_older = work_experience_array[count+1][1] 
                print "End older: {0}".format(end_older)
                
                if (end_newer == end_older): # Pattern 1
                    newer_duration = (end_newer - start_newer).days
                    older_duration = (end_older - start_older).days
                    overlap = min(newer_duration, older_duration)
                    total_overlap += overlap 
                    count += 1
                    print "Pattern 1"
                    print overlap
                
                elif(start_older == start_newer): # Pattern 2
                    overlap = (end_older - start_older).days
                    total_overlap += overlap 
                    count += 1
                    print "Pattern 2"
                    print overlap
                
                elif(start_newer < start_older and end_newer > end_older): # Pattern 3
                    overlap = (end_older - start_older).days
                    total_overlap += overlap 
                    count += 1
                    print "Pattern 3"
                    print overlap
                
                elif(start_newer > start_older and end_newer > end_older and end_older > start_newer): # Pattern 4
                    overlap = (end_older - start_newer).days
                    total_overlap += overlap 
                    count += 1
                    print "Pattern 4"
                    print overlap 
                        
                else:
                    print "No overlap"
                    count += 1
            
            print "Total experience right after overlap: {0}".format(total_experience)
            print "Total overlap right after overlap: {0}".format(total_overlap)
            total_experience = (total_experience - total_overlap)/30
            print "Total experience after overlap: {0}".format(total_experience)
            total_experience = total_experience/12 # Convert to years
            print "Total experience after overlap in years: {0}".format(total_experience )
            
            return total_experience
            
        elif (type(work_experience_list) is dict):
            print "is dict"
            start_date = work_experience_list["exp_from"]
            end_date = work_experience_list["exp_to"]
            if (end_date == "Present"):
                end_date = self.present_date
            start_datetime = datetime.datetime.strptime(start_date, "%m/%Y")
            end_datetime = datetime.datetime.strptime(end_date, "%m/%Y")
            print "Start date: {0}, End_date: {1}".format(start_datetime, end_datetime)
            
            if (start_datetime >= end_datetime):
                return "error"
                print "the dates were flipped or too short time"
            
            total_experience = ((end_datetime - start_datetime).days)/30
            total_experience = total_experience/12 # Convert to years
            
            print "Single experience: {0}".format(total_experience)
            return total_experience
        
        else: 
            return "none"
    
    def work_experience_json(self, user):
        try: 
            work_experience_list = user[0]["experiences"]["experience"]
            
        except:
            return "none"
        
        return json.dumps(work_experience_list, indent=2)
        
    def show_degree(self, user):
        try: 
            ed_history = user[0]["education"]["institution"]
            if(ed_history == ""):
                return "No edu listed"
            elif(type(ed_history) == dict):
                return ed_history["ed_degree"]
            elif(type(ed_history) == list):
                degree_list = []
                for ed_experience in user[0]["education"]["institution"]:
                    if (ed_experience["ed_degree"] == ""):
                        degree_list.append("none")
                    elif (ed_experience["ed_degree"] != ""):
                        degree_list.append(ed_experience["ed_degree"])
                return degree_list
            else:
                return "Error: Some other error"
                
        except:
            return "Error: No education"
        
    def calculate_education(self, user):
        try: 
            ed_history = user[0]["education"]["institution"]
            if(ed_history == ""):
                return "None"
            elif(type(ed_history) == dict):
                if (ed_history["ed_degree"] == ""):
                    highest_education = self.check_highest_education("none")
                elif (ed_history["ed_degree"] != ""):
                    highest_education = self.check_highest_education(ed_history["ed_degree"])
                
                # Check for highest level of degree
                if(highest_education == "No edu listed"): # Don't count anything we can't identify
                    return "None"
                elif(highest_education == "Other"): # Don't count anything we can't identify
                    return "None"
                elif(highest_education == "Doctorate"):
                    return "Doctorate"
                elif(highest_education == "Professional"):
                    return "Professional"
                elif(highest_education == "Master"):
                    return "Master"
                elif(highest_education == "Bachelor"):
                    return "Bachelor"
                elif(highest_education == "Associate"):
                    return "Associate"
                elif(highest_education == "High School"):
                    return "High School"
                else:
                    return "None"
      
            elif(type(ed_history) == list):
                ed_list = []
                for ed_experience in user[0]["education"]["institution"]:
                    if (ed_experience["ed_degree"] == ""):
                        highest_education = self.check_highest_education("none")
                        ed_list.append(highest_education)
                    elif (ed_experience["ed_degree"] != ""):
                        highest_education = self.check_highest_education(ed_experience["ed_degree"])
                        ed_list.append(highest_education)
                # Check for highest level of degree
                if("No edu listed" in ed_list): # Don't count anything we can't identify
                    return "None"
                elif("Other" in ed_list): # Don't count anything we can't identify
                    return "None"
                elif ("Doctorate" in ed_list):
                    return "Doctorate"
                elif("Professional" in ed_list):
                    return "Professional"
                elif("Master" in ed_list):
                    return "Master"
                elif("Bachelor" in ed_list):
                    return "Bachelor"
                elif("Associate" in ed_list):
                    return "Associate"
                elif("High School" in ed_list):
                    return "High School"
                else:
                    return "None"
            else:
                return "Error: Some other error"
                
        except:
            return "None"
    
    def check_highest_education(self, degree):
        # Checks for the highest level of education attained by user (DO NOT CHANGE ORDER!) 
        
        # Professional degree
        if (re.search('d\.c|d\.c\.m|d\.d\.s|d\.m\.d|ll\.b|ll\.m|l\.l\.m|l\.l\.b|j\.d\.|m\.d\.|o\.d\.|d\.o\.|pharm\.d', degree.lower()) 
            or re.search('d\.p\.m|d\.p|pod\.d|m\.div|m\.h\.l|b\.d|ordination|d\.v\.m| law', degree.lower())):
            return "Professional"
        
        # Masters
        elif (re.search('Master|Engineer\'s degree|M\.B\.A', degree)):
            return "Master"

        # Bachelors
        elif (re.search('bachelor|b\.a\.|b\.s\.', degree.lower())):
            return "Bachelor"
        

        # Associate's Degree
        elif (re.search('associate|diploma', degree.lower())):
            return "Associate"

        # PhD
        elif (re.search('doctor', degree.lower())):
            return "Doctorate"

        # High School
        elif (re.search('high school', degree.lower())):
            return "High School"

        elif (re.search('none', degree)):
            return "No edu listed"

        else:
            return "Other" # We couldn't recognize the degree 
    
    def encode_category(self, item, id_list):
        if not(item in id_list):
            id_list.append(item)
        
        category_index = id_list.index(item)
        return category_index
        
    
    def identify_job_category(self, user):
        try:
            all_job_categories = user[0]["dev_job_categories_v2"]["dev_job_categories_v"]
            
        except:
            job_category = "none"
            return job_category
        
        try:
            job_category = all_job_categories[0]["groups"]["group"]["name"] # Returns general job category
        
        except:
            job_category = all_job_categories["groups"]["group"]["name"]
        
        return job_category
    

In [12]:
myObject = UpworkDataFormatter()
myObject.save_all_to_csv()

-----~01b404a60905b11630------
-----~019b9a4166816e3e16------
-----~01c245731f7c607681------
-----~01ccfbbd6370706f4c------
-----~01e3039bd8fb59602c------
-----~017a8ce5f5277a70f8------
-----~01f9b6ea7dce82200c------
-----~01f93508c701a69482------
-----~010ce38df6bfb4bf87------
-----~01cbe2ab4416093a68------
-----~01d63155b42397ba34------
-----~01d459fd13fb47f27c------
-----~012430a7ad47ce70f0------
-----~01a2a1a622f5e86fe4------
-----~01553b7b761bd72b92------
-----~014a2897d520a73b7e------
-----~014fc74e3afb2c9117------
-----~017e7eae60cd64c8fc------
-----~018fd35897afad5062------
-----~01a1cd5d812f168d30------
-----~01c4cbc414bc709812------
-----~011a833163986e26d4------
-----~013ea2e817845372b8------
-----~011c0354c2a11fe2cf------
-----~017103f1940969128f------
-----~01c023ea6a0992bac6------
-----~0131a96dfbd1f6db52------
-----~01f300de8f270c7341------
-----~01334d3bcba0140b7d------
-----~0104c9caa56bfcaa97------
-----~011ab65e13336941fd------
-----~018273718aa6343437------
-----~01

-----~011e0e9cacf66e946d------
-----~0136c57bca5ac9bb10------
-----~01ff69c29a97705f1b------
-----~01e94d54bf26a919d1------
-----~010169ae6c4fa107fa------
-----~0141a1b2f1742f3947------
-----~01206372053d1e5492------
-----~014d84608d20c31dd6------
-----~0114d8050e464c6610------
-----~01c91bc5292d652bdc------
-----~0145a39aa5245b7224------
-----~010f90d89aa2c70956------
-----~01d63a0053755cc7a0------
-----~012d00940c09593a56------
-----~01e63a9d65e498c076------
-----~01c1b8d0c4a52dfd5c------
-----~0111519d994f580ed9------
-----~0179ae7f91a5c34ffd------
-----~01ffc00287ba53e1fd------
-----~01e3a0f707ecc64ebe------
-----~013f7484a708019109------
-----~01e4cded9435d37c67------
-----~0192f966f05cca0edc------
-----~0194f4e044ebc2a04d------
-----~012e9774dbabf0afb7------
-----~018d2945c2b89278ab------
-----~0129c5b1c9ec0eb394------
-----~01e9ca5ad010d5954b------
-----~01f027f982f24e6e16------
-----~013033013bbdc93b46------
-----~015767f77285cd9288------
-----~010e53584ac2d28b88------
-----~01

-----~01c268ec63cc0690a7------
-----~0174381f7b0a88bed7------
-----~019306a985989749d9------
-----~01e1f55bda9a4e822e------
-----~01a764bd4f3f0d1b50------
-----~014bf772545b2ce230------
-----~0155d0cc9e7342d7b0------
-----~018b668480376912de------
-----~01af4710cc98ad010d------
-----~019f2b0a85671b04fc------
-----~010b5eb3ccc26acde6------
-----~017540d588c5017800------
-----~01ec39d9b6e94b6bd5------
-----~0145fb160dc1d92175------
-----~0112c069df1ff8bd66------
-----~0124f38c8fc397c326------
-----~01f083cf163b243399------
-----~017b1318b35e07f77e------
-----~01573e8d28cf78efbf------
-----~01dda36b313649ff93------
-----~0148c79ceca7c9e77e------
-----~01fd05ba5e4d0045d9------
-----~01a128c5f06a9556f7------
-----~017efccb1458c7667a------
-----~01942f6945f4ffa5ef------
-----~01746cf1b469d863e5------
-----~01313daa5d0baec146------
-----~016cf11d24b1e7ae51------
-----~010011e8a3cc82c436------
-----~01d9d6e1a30700807e------
-----~0181f2be0ed4256019------
-----~013ccad852e3f679f4------
-----~01

-----~0118c12263f90e5538------
-----~01f7a1e36b8958b7a3------
-----~0163827bdd3b842602------
-----~0114fa0c12fe8509f6------
-----~012e14d4d0a24cab56------
-----~01f25c67083875b17a------
-----~0120287a1488d50e01------
-----~013da6363d947e9b2e------
-----~0198b3bd2a53347f47------
-----~01774f6ecaec27e45b------
-----~01472c60e48af49dc9------
-----~0156750fa2a13083ad------
-----~01e5245b99fc4014eb------
-----~011cee563bc0f4566c------
-----~018e962d894bb5c49f------
-----~0193816ff44c5f377a------
-----~019c7e7d3ce44e4fdf------
-----~01d242e7be5874b12f------
-----~014feaf3eec05b9e63------
-----~0194d9927fb55928b6------
-----~011499aae9301193e4------
-----~017383cfeb1d9657cb------
-----~01d9eceb5dca4e588d------
-----~012f17cc7b13315948------
-----~01ce5e0cf825712e49------
-----~0108a0862ff3e2f2de------
-----~010672e204ff807223------
-----~01df13fa49f0a4c85b------
-----~01190b88a8dc7f18cd------
-----~01f761a0df1553f2f4------
-----~015060948115e49388------
-----~01f401875bd4fb4d85------
-----~01

-----~013911dfff1d5dad81------
-----~010d8a77cf500c829f------
-----~01b8baa2711339c567------
-----~013b7c118e198eb50d------
-----~016fd501008cf4f088------
-----~01196bc74b3f72c6ce------
-----~014a2890ad397e449a------
-----~012a4d90af3d9fcfef------
-----~011188a2e567ee2e50------
-----~01d8f762a6f06f3cb8------
-----~0172196f92ffb34648------
-----~016292121a1d45fb3a------
-----~0113be77b57574b15a------
-----~01a2424ebbc5af383f------
-----~01aa156aeb5aad665c------
-----~0121df36f292eabe14------
-----~017650c344b3e54979------
-----~0197f3ddd834441f61------
-----~013dd51bb7a1bf2b6f------
-----~016a9c1d86ba69ad26------
-----~01fc880a2e4638d928------
-----~01baa1745339b8c8c1------
-----~010e5f7ddbba65cd49------
-----~01ad09b4101077d831------
-----~01da915cdc3be0495d------
-----~01ed2aa388bb9333f6------
-----~01646b62e3136df81b------
-----~01d27567dcba1a298b------
-----~01bddd7b905acf61c9------
-----~01080c0571f88b65e8------
-----~0173310a16aa7a9a99------
-----~012219a86414688491------
-----~01

...

-----~01ead096236eb8722e------
-----~01cde6ff6a24ee8ce5------
-----~0173ec2764252748d3------
-----~013aa003d99c5489f3------
-----~01e0f25928d92eeca3------
-----~0157cd1ba082a2a0b2------
-----~01aede1fba323b7408------
-----~01dae135deb7fa1162------
-----~01f925c2f41d5ad43c------
-----~0163811c1867dd2ce6------
-----~01b739159c58451d10------
-----~012e1bc07c3dc5d8bb------
-----~01306aaed6c7060f1d------
-----~01d0f43b621cd1f5b2------
-----~011ea0bc9722006105------
-----~010fb102553696b498------
-----~0140cf2285bfcf5446------
-----~01fc6d0a464f87e1ce------
-----~016faf47ba5880b00b------
-----~01cf111f3cce839787------
-----~01b2c8df8289cb43bc------
-----~012f7172462f68add3------
-----~0110be3259984d397a------
-----~0114b8549fba6993d0------
-----~01a682a7d57d7eaee0------
-----~0186d8c118200fb3c8------
-----~0100adcab431a77b19------
-----~014c146a5f4ea1291f------
-----~010cca951b622bdcd0------
-----~012dbf2c4582163105------
-----~015ff62b25faf78ca1------
-----~019757b290e18a6c1e------
-----~01

-----~0124060fb2f6d53df0------
-----~01974ddacecff15d31------
-----~01a6336a0369838c41------
-----~01a3efeac59ab9eb19------
-----~01c716f07eaaf537a1------
-----~01ce8e2b881a241aac------
-----~0191ea2f7462130e5d------
-----~018680ddc9752b5046------
-----~01d9809178da39c68c------
-----~01618aedbcaf76b080------
-----~016f9f8c5971a11ffb------
-----~01c3a554e5467399b2------
-----~0198e31d13d9c0ffd2------
-----~01f4bb79a05dc2b81f------
-----~01e3769b39cbddbc1f------
-----~017f37852c412cddaa------
-----~0177548e3cae4186a5------
-----~01be8c7496f4bbe0fc------
-----~01900f26085c8a8028------
-----~01409aa39d2bb81bca------
-----~016fcf632dad0f7828------
-----~0177fce5e032c2ffdd------
-----~0116ce62637894a2cc------
-----~01dce75044d12ca1e7------
-----~01f6417ca4d5319466------
-----~01e6b77f4a06389b53------
-----~01bb217e88fba7d64d------
-----~0172c03885193dc44d------
-----~0148aa9d113fa61644------
-----~01b4ae92c9444a0f50------
-----~01a8f46e13084b23c4------
-----~01831fbbcba4cc8969------
-----~01

-----~01df987d669469ea2f------
-----~016c596b9cfddcb3e9------
-----~0197f53a091d97c95b------
-----~019aa49853bc30d15c------
-----~0120f07dacab425a1f------
-----~01327c2d914208140e------
-----~01d769945ec18a2ca7------
-----~01e0747aa819080b40------
-----~01945b31b2025636c0------
-----~0127d58251c0df6067------
-----~01025fa3eecf1cee78------
-----~01e34ceecb63ce1a0b------
-----~0168d20dc65d0e3b42------
-----~016aa93711cfd53909------
-----~013f030622465bb774------
-----~0127942b5724280751------
-----~011935ab2f3a9f6f98------
-----~01e3edfb7a3497ce4d------
-----~01e858b61a5666dda1------
-----~01110eb9b9d95bb6be------
-----~0156724fba0c384928------
-----~015e1a67f5d24ea03c------
-----~01270f3451a85634ee------
-----~01dc8635cb37ecfd33------
-----~01d611a8ec1d9c2c5c------
-----~01509f0694570752d2------
-----~012576877d4e283920------
-----~01d8655e841c3edcf4------
-----~0171bad94899a1baf5------
-----~0166f3ae55f2a0aa97------
-----~01bba1f0fe68ff1c23------
-----~01da42b0f4f89a6b34------
-----~01

-----~0163a3ae67f3697374------
-----~014c5ffb19f0c8485a------
-----~015c884a1fb25df515------
-----~01116652218a0d9bf0------
-----~0165c221cff0af133d------
-----~015d78be01e2a38e33------
-----~019ab456aa8f1d60a6------
-----~01dc5d2127acbde2fa------
-----~016fa53b7435c909d6------
-----~0181928ef1d8b43211------
-----~0103b8cbfebedb9de6------
-----~01a43a0326880deaa6------
-----~01d3c443225cd73182------
-----~0125e108ea850e8a43------
-----~01a87c3d3f6e8f00f5------
-----~0160f995e889f02f01------
-----~01a3344851bc8a1096------
-----~01ea10cf818ee33877------
-----~0180be4185b345e17b------
-----~014b77d0cbc78330b8------
-----~0128da3d434d0e342b------
-----~010cf768c07365ccd7------
-----~01bbe08bb02b199db6------
-----~01c773c859448896a3------
-----~01e46a9ccff684fc67------
-----~017ab52f5da2a04b12------
-----~0164120890bb1f1104------
-----~01cc3d6261057b304e------
-----~019be14e8ebfe4d14f------
-----~012477dd4724fb1c6f------
-----~01b1ac7defd165ca94------
-----~01e7172c55ca323a2d------
-----~01

-----~01f21ac8e870242ea3------
-----~01fdbe11eca31e35f1------
-----~0175b792b72f855a1f------
-----~01d8425372cfcfba21------
-----~01acc58e33d32615ef------
-----~01cc7145118db308b3------
-----~015375c622e1a01571------
-----~018ce9c1cf58bfc163------
-----~01d336ff6f09c5627f------
-----~012cc329d46028f269------
-----~01226f0dd79b92ebb2------
-----~015c96a30703911e2c------
-----~019bf42c1a35dc6243------
-----~01b929c24862c05112------
-----~01c59f4ad644947c88------
-----~012a8cbe9a08fd5e7d------
-----~01d12ca1402728ebc2------
-----~01ad2baff71f5db7d4------
-----~01bcf7674ad77b24cd------
-----~01e19fb95a97128be7------
-----~017e2335dc25d4e985------
-----~01d225f4d81403d998------
-----~01ab9b33c8b47a679f------
-----~01c1f3db32b01f06f5------
-----~0194af0c55271b3702------
-----~01261f27b346d4923a------
-----~01818d9055697368d7------
-----~012487a05ecbcd3c8d------
-----~015b51c394fd4c6191------
-----~0140c7e28ea94b0ee4------
-----~0177e142f0fadfbdbc------
-----~01a1ee213578aa6015------
-----~01